* 손실함수 : 결과값을 가장 작게 만드는 함수 가중치 매개변수를 찾을떄 사용 

# 4.1 데이터 학습

## 4.1.1 데이터 주도학습
특징을 추출하고 벡터가 가지는 가중치를 부여

종단간 기계학습 : 처음부터 끝까지 입력에서 결과를 얻는다. 

## 4.1.2 훈련데이터와 시험데이터 
범용능력을 평가하기 위해서 train, test 데이터 분리 

# 4.2 손실함수 
loss function : 최적의 매겨변수 값을 탐색 

평균 제곱 오차, 교차 엔트로피 오차 사용


## 4.2.1 평균 제곱 오차 

mean squared error MSE : 각 원소의 출력(추정) 값과 정답 레이블의 차를 제곱한 후 그 총합 

one hot encoding: 한원소만 1로 하고 그외는 0으로 나타내는 표기범


In [4]:
import numpy as np 

def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [5]:
t = [0,0,1,0,0,0,0,0,0,0]
y = [0.1,0.2,0.7,0.2,0.1,0,0,0,0,0]

print(mean_squared_error(np.array(y), np.array(t)))

t = [0,0,1,0,0,0,0,0,0,0]
y = [0.1,0.2,0.7,0.2,0.1,0,0.8,0,0,0]
print(mean_squared_error(np.array(y), np.array(t)))

0.095
0.415


오답에 가까울 수록 결과가 크다.

## 4.2.2 교차 엔트로피 
cross entroypy CEE : 정답에 해당하는 인덱스만 1 log 를 곱해도 나머진 모두 0이므로 결과에 영향을 주지 않는다. 정답일떄의 출력이 전체 값을 정함

정답에 해당하는 출력이 커질수록 0에 다가가다가 출력이 1일떄 0이 됨, 반대로 정답일때의 추렭이 작아질수록 오차는 커짐

In [7]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))

# np.log(0) 이면 -inf 가 되어 계산을 진행할수 없음 0이 되지 않도록 해줌 

In [13]:
t = [0,0,1,0,0,0,0,0,0,0]
y = [0.1,0.2,0.7,0.2,0.1,0,0,0.1,0.2,0]
print(cross_entropy_error(np.array(y), np.array(t)))

y = [0.1,0.2,0.5,0.2,0.1,0,0.8,0,0,0]
print(cross_entropy_error(np.array(y), np.array(t)))

0.356674801082
0.69314698056


오차가 작은 처번째 추정이 정답일 가능성이 높다고 판단

## 4.2.3 미니배치 학습

모두에 대한 손실 함수의 합을 구함 평균 손실 함수 

일부만 골라서 학습 미니배치 학습 


In [7]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 
import numpy as np
import pickle
# from dataset.mnist import load_mnist
from common.functions import sigmoid, softmax



ModuleNotFoundError: No module named 'dataset'